In [ ]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
%reload_ext autoreload
%autoreload 2

In [ ]:

df_results = load_all_results()

In [ ]:
df_results.sort_values(['model', 'prompt_technique', 'nb_test_run'])

In [ ]:
df_results.groupby(['model', 'prompt_technique', 'nb_test_run']).head(1).sort_values('f1_mean', ascending = False)

In [ ]:
model  = "mistral-7b-v0.1-ft-discussion-10000-Q5_0"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), with_precision = "10_False")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

In [ ]:
r.f1

In [ ]:
from ner.Datasets.Conll2003Dataset import load_conll_dataset
data = load_conll_dataset(cleaned = True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]

In [ ]:
[(d['text'], d['spans']) for d in data if 'Hansenne' in d['text']]